In [3]:

import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from deap import base, creator, tools, algorithms
import random

# Load dataset
data = load_iris()
X = data.data
y = data.target.reshape(-1, 1)

# One-hot encode target
encoder = OneHotEncoder(sparse_output=False)  # Updated for newer scikit-learn versions
y = encoder.fit_transform(y)


# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Neural Network function
def train_and_evaluate(params):
    lr, n_hidden, n_neurons = params
    lr = 10**lr  # scale learning rate
    n_hidden = int(n_hidden)
    n_neurons = int(n_neurons)

    # Create model
    model = keras.Sequential()
    model.add(Dense(n_neurons, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    # Compile and train
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=0, batch_size=16)

    # Evaluate
    _, acc = model.evaluate(X_test, y_test, verbose=0)
    return acc

# Genetic Algorithm setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_lr", random.uniform, -5, -1)  # log scale for learning rate
toolbox.register("attr_n_hidden", random.randint, 1, 3)
toolbox.register("attr_n_neurons", random.randint, 5, 50)
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_lr, toolbox.attr_n_hidden, toolbox.attr_n_neurons), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def eval_individual(ind):
    return train_and_evaluate(ind),

toolbox.register("evaluate", eval_individual)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Run Genetic Algorithm
population = toolbox.population(n=10)
ngen = 5  # Number of generations

result, _ = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=ngen, verbose=True)

# Best solution
best_ind = tools.selBest(population, k=1)[0]
best_acc = train_and_evaluate(best_ind)
print(f"Best individual: {best_ind}, Accuracy: {best_acc}") 


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


gen	nevals
0  	10    
1  	5     
2  	8     
3  	8     
4  	9     
5  	6     
Best individual: [-3.0110646961332113, 2.0, 35.397865938826016], Accuracy: 0.9666666388511658


In [4]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load dataset
data = load_iris()
X = data.data
y = data.target.reshape(-1, 1)

# One-hot encode target
encoder = OneHotEncoder(sparse_output=False)  # Updated for newer scikit-learn versions
y = encoder.fit_transform(y)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Neural Network function
def train_and_evaluate(params):
    lr, n_hidden, n_neurons = params
    lr = 10**lr  # Scale learning rate
    n_hidden = int(n_hidden)
    n_neurons = int(n_neurons)

    # Create model
    model = keras.Sequential()
    model.add(Dense(n_neurons, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    # Compile and train
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=0, batch_size=16)

    # Evaluate
    _, acc = model.evaluate(X_test, y_test, verbose=0)
    return acc

# Grey Wolf Optimizer  (GWO)implementation
def gwo(fitness_function, dim, bounds, n_agents, n_iter):
    # Initialize positions
    positions = np.random.uniform(bounds[:, 0], bounds[:, 1], (n_agents, dim))
    alpha, beta, delta = np.zeros(dim), np.zeros(dim), np.zeros(dim)
    alpha_score, beta_score, delta_score = -np.inf, -np.inf, -np.inf

    # Optimization loop
    for iter in range(n_iter):
        for i in range(n_agents):
            fitness = fitness_function(positions[i])
            if fitness > alpha_score:
                alpha_score, alpha = fitness, positions[i].copy()
            elif fitness > beta_score:
                beta_score, beta = fitness, positions[i].copy()
            elif fitness > delta_score:
                delta_score, delta = fitness, positions[i].copy()

        # Update positions
        a = 2 - iter * (2 / n_iter)  # Linear decrease of exploration
        for i in range(n_agents):
            for d in range(dim):
                r1, r2 = np.random.random(), np.random.random()
                A1, C1 = 2 * a * r1 - a, 2 * r2
                D_alpha = abs(C1 * alpha[d] - positions[i, d])
                X1 = alpha[d] - A1 * D_alpha

                r1, r2 = np.random.random(), np.random.random()
                A2, C2 = 2 * a * r1 - a, 2 * r2
                D_beta = abs(C2 * beta[d] - positions[i, d])
                X2 = beta[d] - A2 * D_beta

                r1, r2 = np.random.random(), np.random.random()
                A3, C3 = 2 * a * r1 - a, 2 * r2
                D_delta = abs(C3 * delta[d] - positions[i, d])
                X3 = delta[d] - A3 * D_delta

                positions[i, d] = (X1 + X2 + X3) / 3

            # Ensure within bounds
            positions[i, d] = np.clip(positions[i, d], bounds[d, 0], bounds[d, 1])

        print(f"Iteration {iter + 1}/{n_iter}, Best Fitness: {alpha_score}")

    return alpha, alpha_score

# Define GWO parameters
n_agents = 10
n_iter = 5
dim = 3  # Number of hyperparameters
bounds = np.array([[-5, -1], [1, 3], [5, 50]])  # Bounds for lr, n_hidden, n_neurons

# Optimize using GWO
def fitness_function(params):
    return train_and_evaluate(params)

best_params, best_score = gwo(fitness_function, dim, bounds, n_agents, n_iter)
print(f"Best parameters: {best_params}, Best accuracy: {best_score}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Iteration 1/5, Best Fitness: 0.9666666388511658
Iteration 2/5, Best Fitness: 0.9666666388511658
Iteration 3/5, Best Fitness: 1.0
Iteration 4/5, Best Fitness: 1.0
Iteration 5/5, Best Fitness: 1.0
Best parameters: [-3.06950381  1.5586003  39.21930145], Best accuracy: 1.0


In [5]:
!pip install pyswarm
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from pyswarm import pso  # PSO library

# Load dataset
data = load_iris()
X = data.data
y = data.target.reshape(-1, 1)

# One-hot encode target
encoder = OneHotEncoder(sparse_output=False)  # Updated for newer scikit-learn versions
y = encoder.fit_transform(y)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Neural Network evaluation function for PSO
def train_and_evaluate(params):
    lr, n_hidden, n_neurons = params
    lr = 10**lr  # Convert from log scale
    n_hidden = int(n_hidden)
    n_neurons = int(n_neurons)

    # Create model
    model = keras.Sequential()
    model.add(Dense(n_neurons, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    # Compile and train
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=0, batch_size=16)

    # Evaluate
    _, acc = model.evaluate(X_test, y_test, verbose=0)

    # Negative accuracy because PSO minimizes the function
    return -acc

# PSO parameter bounds
lb = [-5, 1, 5]  # Lower bounds: learning rate, number of hidden layers, neurons per layer
ub = [-1, 3, 50]  # Upper bounds: learning rate, number of hidden layers, neurons per layer

# Run PSO
best_params, best_loss = pso(train_and_evaluate, lb, ub, swarmsize=10, maxiter=5)

# Output results
best_accuracy = -best_loss  # Convert back to positive accuracy
print(f"Best Hyperparameters: Learning Rate (log): {best_params[0]}, "
      f"Hidden Layers: {int(best_params[1])}, Neurons/Layer: {int(best_params[2])}")
print(f"Best Accuracy: {best_accuracy}")


  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4464 sha256=e95eee2421fa172cf198902bf26c125856bbc7c56d43c27f35f70551a38f87d7
  Stored in directory: /root/.cache/pip/wheels/71/67/40/62fa158f497f942277cbab8199b05cb61c571ab324e67ad0d6
Successfully built pyswarm


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Stopping search: maximum iterations reached --> 5
Best Hyperparameters: Learning Rate (log): -2.4940564725751453, Hidden Layers: 1, Neurons/Layer: 28
Best Accuracy: 1.0


In [7]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import random

# Load dataset
data = load_iris()
X = data.data
y = data.target.reshape(-1, 1)

# One-hot encode target
encoder = OneHotEncoder(sparse_output=False)  # Updated for newer scikit-learn versions
y = encoder.fit_transform(y)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Neural Network function
def train_and_evaluate(params):
    lr, n_hidden, n_neurons = params
    lr = 10 ** float(lr)  # Ensure lr is treated as a float
    n_hidden = int(n_hidden)
    n_neurons = int(n_neurons)

    # Create model
    model = keras.Sequential()
    model.add(Dense(n_neurons, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    # Compile and train
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=0, batch_size=16)

    # Evaluate
    _, acc = model.evaluate(X_test, y_test, verbose=0)
    return acc



# Ant Colony Optimization
class ACO:
    def __init__(self, num_ants, num_iterations, alpha, beta, evaporation_rate, pheromone_init):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.pheromone = pheromone_init

    def optimize(self, bounds):
        best_params = None
        best_acc = 0

        for _ in range(self.num_iterations):
            # Generate solutions (ants)
            ants = []
            for _ in range(self.num_ants):
                params = [self.sample(bound, i) for i, bound in enumerate(bounds)]
                ants.append(params)

            # Evaluate ants
            fitness = [train_and_evaluate(ant) for ant in ants]

            # Update best solution
            for ant, acc in zip(ants, fitness):
                if acc > best_acc:
                    best_params = ant
                    best_acc = acc

            # Update pheromones
            self.update_pheromones(ants, fitness, bounds)

        return best_params, best_acc

    def sample(self, bound, index):
        lower, upper = bound
        pheromone = self.pheromone[index]
        probabilities = (pheromone ** self.alpha) * ((1 / np.arange(lower, upper + 1)) ** self.beta)
        probabilities /= probabilities.sum()
        return np.random.choice(np.arange(lower, upper + 1), p=probabilities)

    def update_pheromones(self, ants, fitness, bounds):
        for i, bound in enumerate(bounds):
            lower, upper = bound
            for j in range(lower, upper + 1):
                self.pheromone[i][j - lower] *= (1 - self.evaporation_rate)  # Evaporate pheromones
                for ant, acc in zip(ants, fitness):
                    if ant[i] == j:
                        self.pheromone[i][j - lower] += acc  # Add pheromone for good solutions

# Hyperparameter bounds
bounds = [(-5, -1), (1, 3), (5, 50)]  # [lr (log scale), n_hidden, n_neurons]

# Initialize pheromone trails
pheromone_init = [np.ones(upper - lower + 1) for lower, upper in bounds]

# ACO Parameters
aco = ACO(num_ants=10, num_iterations=5, alpha=1, beta=2, evaporation_rate=0.1, pheromone_init=pheromone_init)

# Run ACO
best_params, best_acc = aco.optimize(bounds)
print(f"Best parameters: {best_params}, Accuracy: {best_acc}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best parameters: [-1, 1, 5], Accuracy: 1.0


In [8]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load dataset
data = load_iris()
X = data.data
y = data.target.reshape(-1, 1)

# One-hot encode target
encoder = OneHotEncoder(sparse_output=False)  # Updated for newer scikit-learn versions
y = encoder.fit_transform(y)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Neural Network function
def train_and_evaluate(params):
    lr, n_hidden, n_neurons = params
    lr = 10**lr  # scale learning rate
    n_hidden = int(n_hidden)
    n_neurons = int(n_neurons)

    # Create model
    model = keras.Sequential()
    model.add(Dense(n_neurons, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    # Compile and train
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=0, batch_size=16)

    # Evaluate
    _, acc = model.evaluate(X_test, y_test, verbose=0)
    return acc

# Firefly Algorithm
def firefly_algorithm(num_fireflies=10, max_generations=5, alpha=0.2, beta=1, gamma=1):
    # Initialize fireflies with random hyperparameters
    fireflies = np.array([[np.random.uniform(-5, -1),  # Learning rate (log scale)
                           np.random.randint(1, 4),   # Number of hidden layers
                           np.random.randint(5, 50)] # Neurons per layer
                          for _ in range(num_fireflies)])

    fitness = np.array([train_and_evaluate(firefly) for firefly in fireflies])

    for gen in range(max_generations):
        for i in range(num_fireflies):
            for j in range(num_fireflies):
                if fitness[j] > fitness[i]:  # Move firefly i toward firefly j
                    distance = np.linalg.norm(fireflies[i] - fireflies[j])
                    attraction = beta * np.exp(-gamma * distance**2)
                    fireflies[i] += attraction * (fireflies[j] - fireflies[i]) + alpha * np.random.uniform(-1, 1, 3)
                    fireflies[i][1] = np.clip(fireflies[i][1], 1, 3)  # Clip to valid ranges
                    fireflies[i][2] = np.clip(fireflies[i][2], 5, 50)

                    # Update fitness
                    fitness[i] = train_and_evaluate(fireflies[i])

        # Reduce randomness (alpha) over generations
        alpha *= 0.9
        print(f"Generation {gen + 1}, Best Fitness: {np.max(fitness)}")

    best_idx = np.argmax(fitness)
    return fireflies[best_idx], np.max(fitness)

# Run Firefly Algorithm
best_params, best_acc = firefly_algorithm()
print(f"Best hyperparameters: {best_params}, Accuracy: {best_acc}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Generation 1, Best Fitness: 1.0
Generation 2, Best Fitness: 1.0
Generation 3, Best Fitness: 1.0
Generation 4, Best Fitness: 1.0
Generation 5, Best Fitness: 1.0
Best hyperparameters: [-1.89481917  1.13856388  8.06386333], Accuracy: 1.0


In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load dataset
data = load_iris()
X = data.data
y = data.target.reshape(-1, 1)

# One-hot encode target
encoder = OneHotEncoder(sparse_output=False)  # Updated for newer scikit-learn versions
y = encoder.fit_transform(y)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Neural Network function
def train_and_evaluate(params):
    lr, n_hidden, n_neurons = params
    lr = 10**lr  # scale learning rate
    n_hidden = int(n_hidden)
    n_neurons = int(n_neurons)

    # Create model
    model = keras.Sequential()
    model.add(Dense(n_neurons, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    # Compile and train
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=0, batch_size=16)

    # Evaluate
    _, acc = model.evaluate(X_test, y_test, verbose=0)
    return acc

# ABC Algorithm
def abc_algorithm(num_bees=10, max_cycles=5, limit=3):
    # Initialize population of food sources
    food_sources = np.array([[np.random.uniform(-5, -1),  # Learning rate (log scale)
                              np.random.randint(1, 4),    # Number of hidden layers
                              np.random.randint(5, 50)]  # Neurons per layer
                             for _ in range(num_bees)])

    fitness = np.array([train_and_evaluate(food) for food in food_sources])
    trial_counter = np.zeros(num_bees)  # Track stagnation of solutions

    for cycle in range(max_cycles):
        # Employed Bees Phase
        for i in range(num_bees):
            new_food = food_sources[i] + np.random.uniform(-1, 1, 3)  # Generate new solution
            new_food[1] = np.clip(new_food[1], 1, 3)  # Clip to valid range
            new_food[2] = np.clip(new_food[2], 5, 50)
            new_fitness = train_and_evaluate(new_food)
            if new_fitness > fitness[i]:
                food_sources[i] = new_food
                fitness[i] = new_fitness
                trial_counter[i] = 0  # Reset trial counter
            else:
                trial_counter[i] += 1

        # Onlooker Bees Phase
        prob = fitness / np.sum(fitness)
        for _ in range(num_bees):
            selected = np.random.choice(range(num_bees), p=prob)
            new_food = food_sources[selected] + np.random.uniform(-1, 1, 3)
            new_food[1] = np.clip(new_food[1], 1, 3)
            new_food[2] = np.clip(new_food[2], 5, 50)
            new_fitness = train_and_evaluate(new_food)
            if new_fitness > fitness[selected]:
                food_sources[selected] = new_food
                fitness[selected] = new_fitness
                trial_counter[selected] = 0

        # Scout Bees Phase
        for i in range(num_bees):
            if trial_counter[i] > limit:
                food_sources[i] = [np.random.uniform(-5, -1),
                                   np.random.randint(1, 4),
                                   np.random.randint(5, 50)]
                fitness[i] = train_and_evaluate(food_sources[i])
                trial_counter[i] = 0

        print(f"Cycle {cycle + 1}, Best Fitness: {np.max(fitness)}")

    best_idx = np.argmax(fitness)
    return food_sources[best_idx], np.max(fitness)

# Run ABC Algorithm
best_params, best_acc = abc_algorithm()
print(f"Best hyperparameters: {best_params}, Accuracy: {best_acc}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Cycle 1, Best Fitness: 1.0
Cycle 2, Best Fitness: 1.0
Cycle 3, Best Fitness: 1.0
Cycle 4, Best Fitness: 1.0
Cycle 5, Best Fitness: 1.0
Best hyperparameters: [-2.41500388  1.45785269 35.61580025], Accuracy: 1.0


In [6]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load dataset
data = load_iris()
X = data.data
y = data.target.reshape(-1, 1)

# One-hot encode target
encoder = OneHotEncoder(sparse_output=False)  # Updated for newer scikit-learn versions
y = encoder.fit_transform(y)

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Neural Network function
def train_and_evaluate(params):
    lr, n_hidden, n_neurons = params
    lr = 10**lr  # scale learning rate, converting to a valid float (positive)

    # Ensure valid values for number of hidden layers and neurons
    n_hidden = int(np.clip(n_hidden, 1, 3))  # Between 1 and 3 hidden layers
    n_neurons = int(np.clip(n_neurons, 5, 50))  # Between 5 and 50 neurons per layer

    # Create model
    model = keras.Sequential()
    model.add(Dense(n_neurons, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    # Compile and train
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=0, batch_size=16)

    # Evaluate
    _, acc = model.evaluate(X_test, y_test, verbose=0)
    return acc

# Opposition-Based Whale Optimization Algorithm
def obc_woa(pop_size=10, max_iter=5):
    # Initialize population
    population = np.array([[np.random.uniform(-5, -1),  # Learning rate (log scale)
                            np.random.randint(1, 4),    # Number of hidden layers
                            np.random.randint(5, 50)]  # Neurons per layer
                           for _ in range(pop_size)])

    # Evaluate fitness
    fitness = np.array([train_and_evaluate(ind) for ind in population])
    best_idx = np.argmax(fitness)
    best_solution = population[best_idx]
    best_fitness = fitness[best_idx]

    for iteration in range(max_iter):
        a = 2 - iteration * (2 / max_iter)  # Decreasing factor
        for i in range(len(population)):
            r1, r2 = np.random.rand(), np.random.rand()
            A = 2 * a * r1 - a
            C = 2 * r2
            p = np.random.rand()

            if p < 0.5:
                if abs(A) < 1:
                    # Encircle prey
                    D = abs(C * best_solution - population[i])
                    population[i] = best_solution - A * D
                else:
                    # Explore
                    random_ind = population[np.random.randint(0, len(population))]
                    D = abs(C * random_ind - population[i])
                    population[i] = random_ind - A * D
            else:
                # Spiral search
                distance_to_best = abs(best_solution - population[i])
                b = 1
                l = np.random.uniform(-1, 1)
                population[i] = (distance_to_best * np.exp(b * l) *
                                 np.cos(2 * np.pi * l) + best_solution)

            # Clip solutions to valid ranges
            population[i][1] = np.clip(population[i][1], 1, 3)  # Number of hidden layers
            population[i][2] = np.clip(population[i][2], 5, 50)  # Number of neurons per layer

            # Ensure valid range for learning rate
            population[i][0] = np.clip(population[i][0], -5, -1)  # Log scale for learning rate

            # Ensure all values are positive
            population[i] = np.clip(population[i], [-5, 1, 5], [0, 3, 50])

        # Evaluate fitness after updating
        fitness = np.array([train_and_evaluate(ind) for ind in population])

        # Update best solution
        new_best_idx = np.argmax(fitness)
        if fitness[new_best_idx] > best_fitness:
            best_solution = population[new_best_idx]
            best_fitness = fitness[new_best_idx]

        print(f"Iteration {iteration + 1}, Best Fitness: {best_fitness}")

        # Opposition-based learning
        if iteration % 2 == 0:  # Every alternate iteration
            # Create opposite population with the same size as the current population
            opposite_population = -population + 2 * np.random.uniform(-5, 50, size=population.shape)
            opposite_fitness = np.array([train_and_evaluate(ind) for ind in opposite_population])

            # Combine current population and opposite population
            combined_population = np.vstack((population, opposite_population))
            combined_fitness = np.hstack((fitness, opposite_fitness))

            # Select the best solutions
            sorted_indices = np.argsort(combined_fitness)[::-1][:pop_size]
            population = combined_population[sorted_indices]
            fitness = combined_fitness[sorted_indices]

    return best_solution, best_fitness

# Run OBC-WOA
best_params, best_acc = obc_woa()
print(f"Best hyperparameters: {best_params}, Accuracy: {best_acc}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Iteration 1, Best Fitness: 1.0


/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/constant_op.py:108: RuntimeWarning: overflow encountered in cast
  return ops.EagerTensor(value, ctx.device_name, dtype)


Iteration 2, Best Fitness: 1.0
Iteration 3, Best Fitness: 1.0
Iteration 4, Best Fitness: 1.0
Iteration 5, Best Fitness: 1.0
Best hyperparameters: [-1.          1.         23.67450663], Accuracy: 1.0


In [8]:

from docx import Document

def save_results_to_word(results, file_name="optimization_results.docx"):
    # Create a Document object
    doc = Document()

    # Add Title
    doc.add_heading('Optimization Algorithm Comparison', 0)

    # Add a summary or introductory paragraph
    doc.add_paragraph("This document contains the results from various optimization algorithms (Genetic Algorithm, Firefly Algorithm, etc.) "
                      "used to optimize hyperparameters for training a neural network on the Iris dataset.")

    # Add results for each optimization technique
    for method, result in results.items():
        doc.add_heading(f'{method} Results', level=1)
        doc.add_paragraph(f"Best Hyperparameters: {result['best_params']}")
        doc.add_paragraph(f"Best Accuracy: {result['best_acc']}")

    # Save the document
    doc.save(file_name)

results = {
    "Genetic Algorithm": {
        "best_params": [-3.0110646961332113, 2.0, 35.397865938826016],
        "best_acc": 0.9666666388511658
    },
    "Firefly Algorithm": {
        "best_params":[-1.89481917 , 1.13856388 , 8.06386333],
        "best_acc":  1.0
    },
     "GWO": {
        "best_params":[-3.06950381 , 1.5586003 , 39.21930145],
        "best_acc": 1.0
    },
      "PSO": {
        "best_params": [-2.4940564725751453, 1, 28],
        "best_acc": 1.0
    },
      "ACO": {
        "best_params": [-1, 1, 5],
        "best_acc": 1.0
    },
      "ABC": {
        "best_params": [-2.41500388 , 1.45785269 ,35.61580025],
        "best_acc": 1.0
    },
      "OBC-WOA": {
        "best_params": [-1,  1  , 23.67450663],
        "best_acc": 0.96
    }
     
}

# Save results to a Word file
save_results_to_word(results)
print("Results1 have been saved to optimization_results.docx")
8

Results1 have been saved to optimization_results.docx


8